In [7]:
from ipycanvas import Canvas, hold_canvas
import numpy as np

import ipywidgets as widgets



In [8]:
class canvas_widget:
    def __init__(self, gui=None, width=2000, height=1000):
        self.gui = gui
        self._width, self._height = width, height

        self._col_background = "#584f4e"
        self._col_node_header = "#38a8a4"
        self._col_node_selected = "#9dcea6"
        self._col_node_unselected = "#dee7bc"
        
        self._font_size = 40
        self._node_box_size = 200, 100

        self._canvas = Canvas(width=width, height=height)
        self._canvas.fill_style = self._col_background
        self._canvas.fill_rect(0, 0, width, height)
        self._canvas.layout.width = '100%'
        self._canvas.layout.height = 'auto'

        self.objects_to_draw = []
        self.connections = []
        
        self._canvas.on_mouse_down(self.handle_mouse_down)
        self._canvas.on_mouse_move(self.handle_mouse_move)        
        
    def draw_connection(self, path=[0, 1]):
        i_out, i_in = path
        out = self.objects_to_draw[i_out]
        inp = self.objects_to_draw[i_in]

        canvas = self._canvas
        canvas.stroke_style = 'white' 
        canvas.line_width = 3
        canvas.move_to(out.x_out, out.y_out)
        canvas.line_to(inp.x_in, inp.y_in)
        canvas.stroke()
    
    def canvas_restart(self):
        self._canvas.clear()
        self._canvas.fill_style = self._col_background
        self._canvas.fill_rect(0, 0, self._width, self._height)

    def handle_mouse_down(self, x, y):
        if [o for o in self.objects_to_draw if o.selected]:
            [o.set_selected(False) for o in self.objects_to_draw if o.selected]
            return False

        check_bool_pos = list(
            set([check_region.is_selected(x, y) for check_region in self.objects_to_draw])
        )
        if self.gui is None:
            title = 'None'
        else:
            title = self.gui.nodes.value
            
        if len(check_bool_pos) == 1:
            if check_bool_pos[0] == False:
                s = Square_obj(x, y, self, title=title)
                s.set_selected(False)
                s.draw()

            else:
                self.canvas_restart()
                [o.draw() for o in objects_to_draw]

        if len(check_bool_pos) == 0:
            s = Square_obj(x, y, self, title=title)
            s.set_selected(False)
            s.draw()    


    def handle_mouse_move(self, x, y):
        if [o for o in self.objects_to_draw if o.selected]:
            with hold_canvas(self._canvas):
                [o for o in self.objects_to_draw if o.selected][-1].set_x_y(x, y)
                self.canvas_restart()
                [o.draw() for o in self.objects_to_draw]
                for path in self.connections:
                    self.draw_connection(path)

                    
class Square_obj:
    def __init__(self, x, y, canvas_widget, selected=False, title='Title'):
        self.x = x
        self.y = y
        self.width, self.height = canvas_widget._node_box_size
        self.selected = selected
        self.title = title
        self.canvas_widget = canvas_widget
        self.canvas = canvas_widget._canvas
        self.canvas_widget.objects_to_draw.append(self)
        n_obj = len(self.canvas_widget.objects_to_draw)
        if n_obj > 1:
            self.canvas_widget.connections.append([n_obj-2, n_obj-1])

    def set_x_y(self, x_in, y_in):
        self.x = x_in
        self.y = y_in
        
    def draw_title(self, title):
        self.canvas.fill_style = self.canvas_widget._col_background
        self.canvas.font = f'{self.canvas_widget._font_size}px serif'
        x = self.x - (self.width * 0.49)
        y = self.y - (self.height * 0.6),
        self.canvas.fill_text(title, x, y)
        
    def draw_input(self):
        self.canvas.fill_style = self.canvas_widget._col_background
        x = self.x - (self.width * 0.5)
        y = self.y - 0 * (self.height * 0.5)
        self.x_in, self.y_in = x, y        
        self.canvas.fill_style = 'blue'
        self.canvas.fill_arc(x, y, 5, -np.pi/2, np.pi/2)      
        
    def draw_output(self):
        self.canvas.fill_style = self.canvas_widget._col_background
        x = self.x + (self.width * 0.5)
        y = self.y - 0 * (self.height * 0.5)
        self.x_out, self.y_out = x, y
        self.canvas.fill_style = 'red'
        self.canvas.fill_arc(x, y, 5, np.pi/2, 3 * np.pi/2)        

    def draw(self):
        self.canvas.fill_style = self.canvas_widget._col_node_header
        self.canvas.fill_rect(
            self.x - (self.width * 0.5), self.y - (self.height), self.width, self.height
        )
        if self.selected:
            self.canvas.fill_style = self.canvas_widget._col_node_selected
        else:
            self.canvas.fill_style = self.canvas_widget._col_node_unselected
        self.canvas.fill_rect(
            self.x - (self.width * 0.5),
            self.y - (self.height * 0.5),
            self.width,
            self.height,
        )
        self.draw_title(self.title)
        self.draw_input()
        self.draw_output()
        

    def is_selected(self, x_in, y_in):
        x_coord = self.x - (self.width * 0.5)
        y_coord = self.y - (self.height * 0.5)

        if (
            x_in > x_coord
            and x_in < (x_coord + self.width)
            and y_in > y_coord
            and y_in < (y_coord + self.height)
        ):

            self.set_selected(True)
            return True
        else:
            self.set_selected(False)
            return False

    def set_selected(self, state):
        self.selected = state


In [9]:
onto_dic = {}
onto_dic['Utilities'] = {'Murnaghan':[]}
onto_dic['Codes'] = {'Vasp':[], 'Lammps':[]}
onto_dic['Structures'] = {'Bulk':[], 'Surfaces':[]}

class GUI:
    def __init__(self, onto_dic=onto_dic):
        self.canvas_widget = canvas_widget(self)
        self.onto_dic = onto_dic
        
    def draw(self):    
        out = widgets.Output(layout={'border': '1px solid black'})
        with out:
            display(self.canvas_widget._canvas)

        self.modules = widgets.Dropdown(
            options=self.onto_dic.keys(),
            value='Codes',
        #     description='Category:',
            disabled=False,
            layout=widgets.Layout(width='100px')
        )

        self.nodes = widgets.RadioButtons(
            options=self.onto_dic[self.modules.value].keys(),
        #    value='pineapple', # Defaults to 'pineapple'
        #    layout={'width': 'max-content'}, # If the items' names are long
        #     description='Nodes:',
            disabled=False
        )

        run_button = widgets.Button(description="Run")
        add_node_button = widgets.Button(description="Add Node")
        add_connection_button = widgets.Button(description="Add Connection")
        
        self.modules.observe(self.on_value_change, names='value')

        return widgets.VBox([
                      widgets.HBox([add_node_button, add_connection_button]), 
                      widgets.HBox([
                          widgets.VBox([self.modules, self.nodes]), 
                          out])
                       ])        


    def on_value_change(self, change):
        self.nodes.options = onto_dic[self.modules.value].keys()


In [50]:
gui = GUI()
gui.draw()

In [ ]:
from ipycanvas import Canvas

canvas = Canvas(width=4000, height=50)

canvas.font = '32px serif'
canvas.fill_text('Drawing from Python is cool! Drawing from Python is cool!Drawing from Python is cool!', 10, 32)

canvas
# [i for i in canvas.widget_types.items()]

In [ ]:
len(objects_to_draw)

In [ ]:
objects_to_draw[0].x_in

In [33]:
import ryvencore

In [ ]:
import ryvencore as rc
import json
import sys

# f = 'matrices.json'
# try:
#     f = open(fpath)
#     f.close()
# except FileNotFoundError:
#     sys.exit(f'could not open file {fpath}')

# # read project file
# with open(fpath, 'r') as f:
#     project: dict = json.loads(f.read())

# run ryvencore
session = rc.Session()
# session.load(project)

# and now we can manually access all components, for example:
scripts = session.scripts
# flow1 = scripts[0].flow
# my_node = flow1.nodes[-1]
# my_node.update()
scripts